# Proper Scoring Rules

In [1]:
import numpy as np
from scipy.stats import norm

<div align="justify">

[Scoring Rules](https://en.wikipedia.org/wiki/Scoring_rule) assess the quality of probabilistic forecasts, by assigning a numerical score based on the predictive distribution and on the event or value that materializes. A scoring rule is proper if the forecaster maximizes the expected score for an observation drawn from the distribution $F$ if he or she issues the probabilistic forecast $F$, rather than $G \neq F$. It is strictly proper if the maximum is unique. In prediction problems, proper scoring rules encourage the forecaster to make careful assessments and to be honest. In estimation problems, strictly proper scoring rules provide attractive loss and utility functions that can be tailored to the problem at hand ({cite:t}`gnei_sp05`).

</div>

## Continuous Ranked Probability Score

<div align="justify">

The Continuous Ranked Probability Score (CRPS) applies to probabilistic forecasts that take the form of predictive cumulative distribution functions. It generalizes the absolute error and forms a special case of a new and very general type of score, the energy score. Like many other scoring rules, the energy score admits a kernel representation in terms of negative definite functions, with links to inequalities of Hoeffding type, in both univariate and multivariate settings ({cite:t}`gnei_sp05`). 

The CRPS is defined as follows:

$$
\text{CRPS}(F,x) = \int_{-\infty}^{\infty} \left( F(y) - \mathbf{1}\{y \geq x\} \right)^2 \, dy,
$$

where $F$ is the cumulative distribution function (CDF) of the forecast, $x$ is the observed value, and $\mathbf{1}\{y \geq x\}$ is the indicator function, which is 1 if $y \geq x$ and 0 otherwise. 

For a Gaussian (normal) predictive distribution with mean $\mu$ and standard deviation $\sigma$, 
the CRPS can be computed using a closed-form solution:

$$
\text{CRPS}(F, x) = \sigma \left[ \frac{x - \mu}{\sigma} \left( 2\Phi\left( \frac{x - \mu}{\sigma} \right) - 1 \right) + 2\phi\left( \frac{x - \mu}{\sigma} \right) - \frac{1}{\sqrt{\pi}} \right], 
$$

where $\Phi$ is the CDF of the standard normal distribution and $\phi$ is the probability density function (PDF) of the standard normal distribution.

</div>

In [4]:
def crps(y_true, y_pred):
    residuals = y_true - y_pred
    y_std = np.std(residuals)
    crps_values = []
    for i in range(len(y_true)):
        mu = y_pred[i]
        sigma = y_std
        obs = y_true[i]
        term1 = np.abs(obs - mu)
        term2 = sigma * (1 / np.sqrt(np.pi))
        crps_value = term1 + term2 - 2 * sigma * norm.cdf((obs - mu) / sigma)
        crps_values.append(crps_value)
    return np.mean(crps_values)

## Brier Score

<div align="justify">

The [Brier Score](https://en.wikipedia.org/wiki/Brier_score) (Quadratic Score) is a strictly proper scoring rule which measures the accuracy of probabilistic predictions, 

$$
S_{\text{Brier}}= \frac{1}{N} \sum_{i=1}^N (p_i-y_i)^2.
$$

</div>

In [ ]:
def brier_score(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean((y_pred - y_true) ** 2)

<div align="justify">

```{note}
The Brier score can be thought of as a cost function. Thus lower values indicate better performance. 
In its most common formulation, it takes on a value between 0 and 1, 
since this is the square of the largest possible difference between a predicted probability and the actual outcome. 
The Brier score is appropriate for binary and categorical outcomes that can be structured as true or false, 
but it is inappropriate for ordinal variables which can take on three or more values.
```

</div>

## Logarithmic Score

<div align="justify">

The Logarithmic (Log) Score is also often used in the context of probability forecasting, 

$$
S_{\text{Log}} = -\sum_{i=1}^N y_i \log(p_i).
$$

</div>

In [2]:
def logarithmic_score(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.sum(y_true * np.log(y_pred))

## Log Loss

<div align="justify">

It measures the performance of a classification model where the prediction is a probability value between 0 and 1, 

$$
L = -\frac{1}{N} \sum_{i=1}^N \left(y_i \log(p_i) + (1-y_i) \log(1-p_i) \right).
$$

</div>

In [ ]:
def log_loss(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

<div align="justify">

```{hint}
The Log Loss, Logarithmic Loss, Logistic Loss, and Cross-entropy Loss are used interchangeably. 
```

</div>

## Mean Square Error

<div align="justify">

In statistics, the [mean squared error (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error) or mean squared deviation (MSD) of an estimator (of a procedure for estimating an unobserved quantity) measures the average of the squares of the errors,

$$
\text{MSE} = \frac{1}{N} \sum_{i=1}^N (\hat{y_i}-y_i)^2.
$$

</div>

In [ ]:
def mean_squared_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean((y_pred - y_true) ** 2)

<div align="justify">

```{note}
The Mean Squared Error is a more general measure of accuracy, used primarily in regression tasks. 
It measures the average of the squares of the differences between predicted values and actual values. 
While the Brier Score is used specifically for probabilistic predictions in binary classification tasks. 
It measures the mean squared difference between predicted probabilities and the actual binary outcomes.
```

</div>

## $R^2$

<div align="justify">

R-squared (coefficient of determination) measures the proportion of the variance in the dependent variable that is predictable from the independent variables, 

$$
R^2 = 1 - \frac{\sum_{i=1}^N (y_i-\hat{y_i})^2}{\sum_{i=1}^N (y_i-\bar{y_i})^2}.
$$

</div>

In [ ]:
def r_squared(y_true, y_pred):
    error = y_pred - y_true
    sse = (error ** 2).sum()
    tss = ((y_true - y_true.mean()) ** 2).sum()    
    return 1 - sse / tss

<div align="justify">

```{note}
$N$ denotes the number of observations or data points in the dataset.
```

</div>